In [2]:
import numpy as np
import math
import pandas as pd

In [3]:
def load_data(l):
    data=np.loadtxt(l,delimiter=",",skiprows=1)
    X = data[:,1:]
    Y = data[:,0]
    return (X,Y)
def split(X,Y,s):
    m=X.shape[0]
    a=math.ceil(s*m)
    return(X[:a],Y[:a],X[a:],Y[a:])

In [4]:
def distance(x,X):
    X_new = np.sum(X**2, axis = 1)
    x_new = np.sum(x**2, axis = 1)
    X_x_dot = 2*(x @ X.T)
    dist = X_new - X_x_dot + x_new.reshape((x_new.shape[0],1))
    return dist
def nearest_class(x):
    class_, occurence = np.unique(x, return_counts=True)        
    y_index = class_[np.argmax(occurence)]
    return y_index
def classifying(Y_train,dist,k):
    y_i=np.argsort(dist)[:,:k]
    y_temp=Y_train[y_i]
    y=np.apply_along_axis(nearest_class, axis=1,arr=y_temp)
    return y

In [5]:
def accuracy(Y_pred,Y_cv):
    print('Train Accuracy: %f'%(np.mean(Y_pred == Y_cv) * 100))    
def f1score(classes, y_train,y_predicted):
    con_matrix = np.zeros((classes,classes),dtype=int)
    for i, k in zip(y_train, y_predicted):
        con_matrix[int(i),int(k)] += 1
    df = pd.DataFrame(con_matrix)
    precision = np.zeros(classes)
    recall = np.zeros(classes)
    f1 = np.zeros(classes)
    for i in range(classes):
        t_p = con_matrix[i, i]
        f_p = np.sum(con_matrix[:, i]) - t_p
        f_n = np.sum(con_matrix[i, :]) - t_p
        precision[i] = t_p / (t_p + f_p) if (t_p + f_p) != 0 else 0
        recall[i] = t_p / (t_p + f_n) if (t_p + f_n) != 0 else 0
        f1[i] = 2 * (precision[i] * recall[i]) / (precision[i] + recall[i]) if (precision[i] + recall[i]) != 0 else 0
    f1score = np.mean(f1)
    return f1score, df

In [6]:
X,Y = load_data('Classification_train.csv')
X_new = X/255

In [7]:
print(f"max in original :{np.max(X)}, min in original :{np.min(X)}")   
print(f"max in normalized :{np.max(X_new)}, min in original :{np.min(X_new)}")

max in original :255.0, min in original :0.0
max in normalized :1.0, min in original :0.0


In [8]:
X_train,Y_train,X_cv,Y_cv = split(X,Y,0.83)

In [9]:
dist = distance(X_cv,X_train)
neighbours = 1
p_cv = classifying(Y_train,dist,neighbours)

In [10]:
a=accuracy(p_cv,Y_cv) 

f1_score, confusion_matrix = f1score(10,Y_cv,p_cv)
print(f"f1score: {f1_score}")
print()
print("confusion_matrix")
confusion_matrix

Train Accuracy: 98.352941
f1score: 0.9834858111319988

confusion_matrix


,0,1,2,3,4,5,6,7,8,9
0,477,14,0,0,0,0,0,0,0,0
1,4,504,0,0,0,0,0,0,0,0
2,2,0,511,1,1,0,0,0,0,0
3,4,0,0,508,1,0,0,4,0,0
4,1,0,0,1,512,1,1,1,0,2
5,0,0,0,4,2,521,1,0,0,1
6,0,0,2,1,0,0,505,5,0,4
7,0,0,0,7,1,0,8,488,0,1
8,6,0,0,0,0,0,0,0,474,1
9,0,0,0,0,0,0,2,0,0,516


In [18]:
X_test,Y_id = load_data('Classification_test.csv')
X_test_n = X_test
dist_n = distance(X_test_n,X_train)
neighbours = 1

test = classifying(Y_train,dist_n,neighbours)

np.savetxt('knn_test_prediction.csv', test, delimiter=',', header='Test target', comments='')